In [ ]:
import numpy as np

def calculate_snr(signal, noise):
    # Calculate the power of the signal
    signal_power = np.sum(np.square(signal))
    
    # Calculate the power of the noise
    noise_power = np.sum(np.square(noise))
    
    # Calculate SNR in decibels (dB)
    snr_db = 10 * np.log10(signal_power / noise_power)
    
    return snr_db

# Example signal and noise arrays (replace these with your actual data)
signal_array = np.array([1, 2, 3, 4, 5])
noise_array = np.array([0.5, 0.2, 0.3, 0.1, 0.4])

# Calculate SNR
snr_result = calculate_snr(signal_array, noise_array)
print("Signal-to-Noise Ratio (SNR): {:.2f} dB".format(snr_result))

In [ ]:
import librosa
import math

def PitchPorPersona(y,sr):
    f0, voiced_flag, voiced_probs=librosa.pyin(
        y,
        fmin=librosa.note_to_hz('C2'),
        fmax=librosa.note_to_hz('C7'),
        sr=sr)
    f0 = [x for x in f0 if not isinstance(x, float) or not math.isnan(x)]
    f0 = [i for i in f0 if i != 0]
    return f0

In [ ]:
import numpy as np

def Promediador(x):
    return np.mean(x)

In [ ]:
import pyloudnorm as pyln
import librosa 

def LoudnessPorPersona(y,sr):
    meter = pyln.Meter(sr)
    loudness = meter.integrated_loudness(y)
    return loudness

In [ ]:
import pandas as pd

def CalculadoraParametrosAcusticos(path_carpeta, nombre, cant_audios, extension, cant_basesdedatos):
    df = pd.DataFrame(columns=['Pitch', 'Loudness','Tipo'])
    for j in range(cant_basesdedatos):
        for i in range(cant_audios):
            path = f'{path_carpeta}/{nombre[j]}{i}.{extension[j]}'    
            y,sr = librosa.load(path)
            p=Promediador(PitchPorPersona(y,sr))
            l=Promediador(LoudnessPorPersona(y,sr))
            if j==0:
                variable_tipo = 'Normal' 
            elif j==1:
                variable_tipo = 'Pitched'
            df = df.append({'Pitch':p, 'Loudness':l,'Tipo':variable_tipo},ignore_index=True)
    return df

In [ ]:
path_carpeta = 'C:/Users/Asus/OneDrive/Escritorio/MapaDeVoces/Prototipos/Prototipo2/Audios'
nombre = ['HolaComoEstas','HolaComoEstasPitched']
extension = ['opus','wav']
cant_audios = 56
cant_basesdedatos = 2

BaseDeDatos=CalculadoraParametrosAcusticos(path_carpeta,nombre,cant_audios,extension,cant_basesdedatos)

In [ ]:
import umap
import seaborn as sns
from sklearn.preprocessing import StandardScaler

reducer = umap.UMAP()
pitch = BaseDeDatos['Pitch']
pitch = np.asarray(pitch)
pitch = pitch.reshape(-1, 1)
scaled_pitch = StandardScaler().fit_transform(pitch) # estandariza los valores

embedding = reducer.fit_transform(pitch) 

In [ ]:
import plotly.express as px

fig = px.scatter(
    embedding, x=0, y=1,
    color=BaseDeDatos.Tipo, labels={'color': 'Tipo'}
)

fig.update_layout(
    autosize=True,
    width=800,
    height=500,
)
fig.show()